# 「RPCを用いた分散パイプライン並列化」

【原題】Distributed Pipeline Parallelism Using RPC

【原著】[Shen Li](https://mrshenli.github.io/)

【元URL】https://pytorch.org/tutorials/intermediate/dist_pipeline_parallel_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年12月04日

【チュトーリアル概要】

前提知識
- [PyTorch Distributedの概要](https://pytorch.org/tutorials/beginner/dist_overview.html)（日本語版6_1）
- [シングルマシン環境におけるモデル並列訓練](https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html)（日本語版6_2）
- [分散RPCフレームワーク入門](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html)（日本語版6_5）
- RRefで役に立つ関数：[RRef.rpc_sync()](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.RRef.rpc_sync)、[RRef.rpc_async()](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.RRef.rpc_async)、及び [RRef.remote()](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.RRef.remote)

<br>

本チュートリアルでは、Resnet50のモデルに対して、[torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) APIで分散パイプライン並列化を実装する方法を解説します。

なお分散パイプライン並列化は、[シングルマシン環境におけるモデル並列訓練](https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html)で解説したマルチGPUのパイプラインが並列分散化した内容と捉えることが可能です。




**注意**

- 本チュートリアルにはPyTorch v1.6.0以上が必要になります。

- 本チュートリアルのソースコードは、[pytorch/examplesのこちら](https://github.com/pytorch/examples/tree/master/distributed/rpc/pipeline) にて確認できます。

## はじめに

2つ前のチュートリアル、[分散RPCフレームワーク入門](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html)（日本語版6_5） では、`torch.distributed.rpc` を使い、分散モデル並列化を行ってRNNモデルを実装する方法を示しました。

その際は、一つのGPUで `EmbeddingTable` を管理してコードを正常に動作させていました。

しかし、モデルが複数のGPUに存在する場合には、すべてのGPUの稼働率を増やすため、もう少し手順が必要になります。

パイプライン並列化は、このようなケースにおいて役立つ枠組みの一つとなります。

本チュートリアルでは、[シングルマシン環境におけるモデル並列訓練](https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html)（日本語版6_2） のチュートリアルでも使用された `ResNet50` を例に使用します。

また同様に、ResNet50 のモデルを2片に分割し、入力バッチも複数の固まりに分けた上で、パイプライン化された手法で2片のモデルに与えます。

「シングルマシン環境におけるモデル並列訓練」で示した手法との違いは、CUDAのストリームを用いて実行を並列化する代わりに、本チュートリアルでは非同期RPCを呼び出している点です。

そのため、本チュートリアルで解説する内容は、コンピューティング・マシンの境界を超えても動作します。

これから4つのステップで、実装方法を解説します。

## Step 1: ResNet50 モデルの分割

始めに、モデルを分割した状態で、 `ResNet50` を実装する準備段階を行います。


下記のコードは、[torchvisionのResNetの実装](https://github.com/pytorch/vision/blob/7c077f6a986f05383bcb86b535aedb5a63dd5c4b/torchvision/models/resnet.py#L124)から借用しています。

`ResNetBase` モジュールは、分割された2片のResNetに共通する構成要素と属性変数を含んでいます。

In [ ]:
import threading

import torch
import torch.nn as nn

from torchvision.models.resnet import Bottleneck

num_classes = 1000


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class ResNetBase(nn.Module):
    def __init__(self, block, inplanes, num_classes=1000,
                groups=1, width_per_group=64, norm_layer=None):
        super(ResNetBase, self).__init__()

        self._lock = threading.Lock()
        self._block = block
        self._norm_layer = nn.BatchNorm2d
        self.inplanes = inplanes
        self.dilation = 1
        self.groups = groups
        self.base_width = width_per_group

    def _make_layer(self, planes, blocks, stride=1):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if stride != 1 or self.inplanes != planes * self._block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * self._block.expansion, stride),
                norm_layer(planes * self._block.expansion),
            )

        layers = []
        layers.append(self._block(self.inplanes, planes, stride, downsample, self.groups,
                                self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * self._block.expansion
        for _ in range(1, blocks):
            layers.append(self._block(self.inplanes, planes, groups=self.groups,
                                    base_width=self.base_width, dilation=self.dilation,
                                    norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def parameter_rrefs(self):
        return [RRef(p) for p in self.parameters()]

これでモデルを定義する準備ができました。

以下で実装します。

以下のコンストラクターでは、すべてのResNet50の層を2つの部分に分け、それぞれの部分を所与のデバイスに移動しているだけです。

どちらのモデル片の `forward` 関数も、入力データのRRefを引数に取り、データをローカルで与え、その後目的のデバイスに移動させています。

そして、すべての層を入力に適用した後は、出力をCPUに移して返しています。

これは、呼び出し元と呼び出し先でデバイスの数が一致していない場合のデバイスエラーを避ける目的で、”テンソルがCPU上に存在すること”をRPC APIが必要としているために必要な操作となります。

In [ ]:
class ResNetShard1(ResNetBase):
    def __init__(self, device, *args, **kwargs):
        super(ResNetShard1, self).__init__(
            Bottleneck, 64, num_classes=num_classes, *args, **kwargs)

        self.device = device
        self.seq = nn.Sequential(
            nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False),
            self._norm_layer(self.inplanes),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            self._make_layer(64, 3),
            self._make_layer(128, 4, stride=2)
        ).to(self.device)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x_rref):
        x = x_rref.to_here().to(self.device)
        with self._lock:
            out =  self.seq(x)
        return out.cpu()


class ResNetShard2(ResNetBase):
    def __init__(self, device, *args, **kwargs):
        super(ResNetShard2, self).__init__(
            Bottleneck, 512, num_classes=num_classes, *args, **kwargs)

        self.device = device
        self.seq = nn.Sequential(
            self._make_layer(256, 6, stride=2),
            self._make_layer(512, 3, stride=2),
            nn.AdaptiveAvgPool2d((1, 1)),
        ).to(self.device)

        self.fc =  nn.Linear(512 * self._block.expansion, num_classes).to(self.device)

    def forward(self, x_rref):
        x = x_rref.to_here().to(self.device)
        with self._lock:
            out = self.fc(torch.flatten(self.seq(x), 1))
        return out.cpu()

## Step 2: ResNet50のモデル片を一つのモジュールへ

次に、2片のモデルを結合して `DistResNet50` モジュールを作成し、パイプライン並列のロジックを実装します。

コンストラクターでは、2つの `rpc.remote` を呼び出し、2片のモデルをそれぞれ異なるRPCのワーカー上に配置します。

そして、フォワードパスで参照できるように、2つのモデルの部分へのRRefを保持しておきます。




`forward`関数では、入力バッチを複数のマイクロバッチに分割し、これらのマイクロバッチをパイプライン化された方法で2つのモデルの部分に与えます。

最初に、`rpc.remote` を呼び出して最初のモデル片をマイクロバッチに適用し、その後、返ってきた中間出力のRRefを2つ目のモデル片にフォワードします。

そして、すべてのマイクロ出力の `Future` を集め、ループ後にすべての `Future` が完了するまで待機します。



なお、`remote()` と `rpc_async()`　は、返り値を即時に返し、非同期に処理を実行する点に注意してください。

したがって、ループ全体でブロックは行われず、複数のRPCが同時に起動します。

2つのモデルの部分における一つのマイクロバッチの実行順序は、中間出力 `y_rref` によって保持されています。

なお、マイクロバッチ間での実行順序は問題になりません。

最後に、`forward` 関数は、すべてのマイクロバッチの出力を単一の出力テンソルに結合して、値を返します。

`parameter_rrefs` 関数は、分散オプティマイザーの構築を簡略化するための関数ですが、この関数は後ほど使用します。

In [ ]:
class DistResNet50(nn.Module):
    def __init__(self, num_split, workers, *args, **kwargs):
        super(DistResNet50, self).__init__()

        self.num_split = num_split

        # ResNet50の最初の部分を workers[0] に配置します。
        self.p1_rref = rpc.remote(
            workers[0],
            ResNetShard1,
            args = ("cuda:0",) + args,
            kwargs = kwargs
        )

        # ResNet50の2つ目の部分を workers[1] に配置します。
        self.p2_rref = rpc.remote(
            workers[1],
            ResNetShard2,
            args = ("cuda:1",) + args,
            kwargs = kwargs
        )

    def forward(self, xs):
        out_futures = []
        for x in iter(xs.split(self.split_size, dim=0)):
            x_rref = RRef(x)
            y_rref = self.p1_rref.remote().forward(x_rref)
            z_fut = self.p2_rref.rpc_async().forward(y_rref)
            out_futures.append(z_fut)

        return torch.cat(torch.futures.wait_all(out_futures))

    def parameter_rrefs(self):
        remote_params = []
        remote_params.extend(self.p1_rref.remote().parameter_rrefs().to_here())
        remote_params.extend(self.p2_rref.remote().parameter_rrefs().to_here())
        return remote_params

## Step 3: 訓練ループの定義

モデルを定義した後は、訓練ループを実装しましょう。

専用の"マスター"ワーカーを使用して、ランダムな入力とラベルを準備し、分散バックワードパスと分散オプティマイザーステップを制御します。

最初に `DistResNet50` モジュールのインスタンスを作成します。

`DistResNet50` のインスタンス化の際には、各バッチにおけるマイクロバッチの数を指定し、2つのRPCワーカーの名前（例："worker1" と "worker2"）も渡します。



そして、損失関数を定義し、`parameter_rrefs()` を使用してパラメーターの `RRef` のリストを取得した上で、 `DistributedOptimizer` を作成します。

残りの部分は、`dist_autograd` を使用してバックワードを起動します。

`context_id` をバックワードとオプティマイザーの`step()` の両方に渡している点を除けば、主な訓練ループの手順は、通常のローカルでの訓練と良く似た内容です。

In [ ]:
import torch.distributed.autograd as dist_autograd
import torch.optim as optim
from torch.distributed.optim import DistributedOptimizer

num_batches = 3
batch_size = 120
image_w = 128
image_h = 128


def run_master(num_split):
    # 2つのモデルの部分を、それぞれ worker1 と worker2 に配置します。
    model = DistResNet50(num_split, ["worker1", "worker2"])
    loss_fn = nn.MSELoss()
    opt = DistributedOptimizer(
        optim.SGD,
        model.parameter_rrefs(),
        lr=0.05,
    )

    one_hot_indices = torch.LongTensor(batch_size) \
                        .random_(0, num_classes) \
                        .view(batch_size, 1)

    for i in range(num_batches):
        print(f"Processing batch {i}")
        # ランダムな入力とラベルの生成
        inputs = torch.randn(batch_size, 3, image_w, image_h)
        labels = torch.zeros(batch_size, num_classes) \
                    .scatter_(1, one_hot_indices, 1)

        with dist_autograd.context() as context_id:
            outputs = model(inputs)
            dist_autograd.backward(context_id, [loss_fn(outputs, labels)])
            opt.step(context_id)

## Step 4: RPCプロセスの起動

最後に、下記のコードはすべてのプロセスを対象にした関数を示しています。

主なロジックは、`run_master` にて定義されています。

なお、ワーカーは受動的にマスターからの指令を待機し、単に `init_rpc` と `shutdown` を実行しますが、この `shutdown` はデフォルト動作として、関わっているすべてのRPCが終了するまで処理を待ち、ブロックしてくれます。

In [ ]:
import os
import time

import torch.multiprocessing as mp


def run_worker(rank, world_size, num_split):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    options = rpc.TensorPipeRpcBackendOptions(num_worker_threads=128)

    if rank == 0:
        rpc.init_rpc(
            "master",
            rank=rank,
            world_size=world_size,
            rpc_backend_options=options
        )
        run_master(num_split)
    else:
        rpc.init_rpc(
            f"worker{rank}",
            rank=rank,
            world_size=world_size,
            rpc_backend_options=options
        )
        pass

    # すべての rpc が終了するまでブロック
    rpc.shutdown()


if __name__=="__main__":
    world_size = 3
    for num_split in [1, 2, 4, 8]:
        tik = time.time()
        mp.spawn(run_worker, args=(world_size, num_split), nprocs=world_size, join=True)
        tok = time.time()
        print(f"number of splits = {num_split}, execution time = {tok - tik}")

下記の出力は、各バッチの分割数を増やすことで実現できる高速化を示しています。

```
$ python main.py
Processing batch 0
Processing batch 1
Processing batch 2
number of splits = 1, execution time = 16.45062756538391
Processing batch 0
Processing batch 1
Processing batch 2
number of splits = 2, execution time = 12.329529762268066
Processing batch 0
Processing batch 1
Processing batch 2
number of splits = 4, execution time = 10.164430618286133
Processing batch 0
Processing batch 1
Processing batch 2
number of splits = 8, execution time = 9.076049566268921
```